# Text Mining Project on Korea Herald

# Load Data

In [1]:
import json
import pandas as pd
import gzip
import numpy as np

data_path = '../data/koreaherald_1517_#.json.gz'

df_data = pd.DataFrame()

for i in range(8):
  p = data_path.replace('#',str(i))
  with gzip.open(p,'rb') as f:
    data = pd.DataFrame.from_dict(json.load(f))
  df_data = df_data.append(data,ignore_index=True)

# clean up column names
df_data = df_data.rename(columns={" author": "author",
                        " time": "time",
                        " description": "description",
                        " body": "body",
                        " section": "section",
                       })
# preview data
print('Number of docs: {}'.format(df_data.shape[0]))
df_data.dtypes

Number of docs: 23769


title          object
author         object
time           object
description    object
body           object
section        object
dtype: object

# Pre-Processing

Here we apply:
- tokenisation
- lemmatisation
- normalisation

(optional) Bigrams

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
whitelist = {'US':'United States'}

def tokenise_pipeline(doc):
  for k,v in whitelist:
    doc = doc.replace(k,v)
  doc = doc.lower()  # Convert to lowercase.
  tokens = tokenizer.tokenize(doc) # split into words
  # TODO: remove stopwords
  tokens = [token for token in tokens if not token.isnumeric()] # remove numbers
  tokens = [token for token in tokens if token not in stop_words]
  tokens = [token for token in tokens if len(token) > 2] # remove words of only 1 letter
  tokens = [lemmatizer.lemmatize(token) for token in tokens] # lemmatisation
  return tokens

In [3]:
df_data['body_tokenised'] = df_data['body'].apply(tokenise_pipeline)

In [4]:
from gensim.models import Phrases

bigrams = Phrases(df_data['body_tokenised'].values, min_count=20) # keeps only phrases that appear >= 20 times in corpus.

def add_bigrams(doc):
  for token in bigrams[doc]:
    if '_' in token:
      doc.append(token)
  return doc

df_data['body_tokenised'] = df_data['body_tokenised'].apply(add_bigrams)

In [5]:
from gensim.corpora import Dictionary

dictionary = Dictionary(df_data['body_tokenised'].values)
dictionary.filter_extremes(no_below=20, no_above=0.5) # filter words that occur in less than 20 docs or more than 50% docs

print('Number of unique tokens: {}'.format(len(dictionary)))
print('Number of documents: {}'.format(len(df_data['body_tokenised'].values)))

Number of unique tokens: 16614
Number of documents: 23769


In [6]:
# from collections import Counter

# corpus = [word for doc in df_data['body_tokenised'] for word in doc]
# vocab = list(set(corpus))
# dictionary = pd.DataFrame(vocab,columns=['word'])
# dictionary['doc_count'] = dictionary['word'].apply(lambda w: len(df_data[df_data['body_tokenised'].apply(lambda doc: w in doc)]))
# print('Number of unique tokens: {}'.format(len(dictionary)))

In [7]:
# # filter words that occur in less than 20 docs or more than 50% docs
# dictionary = dictionary[(dictionary['doc_count'] < 20) | (dictionary['doc_count'] > 1500)]
# vocab = dictionary.word.values
# df_data['body_tokenised'] = df_data['body_tokenised'].apply(lambda doc : list(filter(lambda w: w in vocab,doc)))
# print('Number of unique tokens: {}'.format(len(dictionary)))

In [8]:
# df_data['body_tokenised'] = df_data['body_tokenised'].apply(lambda doc : ''.join(doc))
# corpus = [doc for doc in df_data['body_tokenised']]

# Vectorization

In [9]:
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.pipeline import Pipeline
# import numpy as np

# pipe = Pipeline([('count', CountVectorizer(vocabulary=vocab)),
#                  ('tfid', TfidfTransformer())]).fit(corpus)

# df_data['body_vector'] = df_data['body_tokenised'].apply(lambda doc : np.squeeze(np.asarray(pipe.transform([doc]).todense())))

In [10]:
# BOW vectorisation
# TFIDF vectorisation
from gensim.models import TfidfModel

corpus = [dictionary.doc2bow(doc) for doc in df_data['body_tokenised']]
tfidf = TfidfModel(corpus)
df_data['body_bow'] = df_data['body_tokenised'].apply(dictionary.doc2bow)
df_data['body_tfidf'] = df_data['body_bow'].apply(lambda doc : tfidf[doc])

# Group By Year

In [11]:
import copy

df_2015 = copy.deepcopy(df_data[df_data['time'].str.contains('2015')])
df_2016 = copy.deepcopy(df_data[df_data['time'].str.contains('2016')])
df_2017 = copy.deepcopy(df_data[df_data['time'].str.contains('2017')])

# LDA

In [12]:
from gensim.models import LdaModel

temp = dictionary[0] # to 'load' dictionary
id2word = dictionary.id2token

# parameters
feature = 'body_bow' # lda uses bow
corpus = df_2015[feature].values
num_topics = 20

model = LdaModel(
  corpus = corpus,
  id2word = id2word,
  chunksize = 2000,
  alpha = 'auto',
  eta = 'auto',
  iterations = 400,
  num_topics = num_topics,
  passes = 20,
  eval_every = None
  )

## LDA Visualisation

In [13]:
# ! pip3 install pyldavis

In [14]:
import pyLDAvis
from pyLDAvis import gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(model, corpus, dictionary)

ModuleNotFoundError: No module named 'pyLDAvis'

In [ ]:
# The larger the bubble, the more prevalent is that topic.
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

## Assign Topics to Docs

In [ ]:
def argmax(ls):
  return max(ls, key = lambda item: item[1])


df_2015['topic'] = df_2015[feature].apply(lambda x : argmax(model.get_document_topics(x))[0])
df_2015['topic_confidence'] = df_2015[feature].apply(lambda x : argmax(model.get_document_topics(x))[1])

# Cluster Analysis

## Metric 1: Number of Documents

In [ ]:
df_2015.topic.value_counts()

## Metric 2: Duration of Reporting

In [ ]:
df_2015[df_2015['topic'] == 1].time.max()

In [ ]:
from datetime import datetime

report_duration = pd.DataFrame(columns=['topic','duration'])

for i in range(num_topics):
  df = df_2015[df_2015['topic'] == i]
  max_t = datetime.strptime(df.time.max(), "%Y-%m-%d %H:%M:%S").date()
  min_t = datetime.strptime(df.time.min(), "%Y-%m-%d %H:%M:%S").date()
  report_duration = report_duration.append(pd.DataFrame({'topic': i, 'duration': (max_t - min_t)}, index=[i]))
  
report_duration.sort_values('duration',ascending=False)

## Metric 3: Entity Relations

In [ ]:
import re
import spacy
from spacy import displacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
import networkx as nx
import matplotlib.pyplot as plt

nlp = spacy.load('en_core_web_sm')

In [ ]:
def get_entities(sent):
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  for tok in nlp(sent):
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [ ]:
topic_docs = df_2015[df_2015['topic'] == 1].title.values

entity_pairs = []

for doc in topic_docs:
  entity_pairs.append(get_entities(doc))
  
entity_pairs[10:20]

# Get n_topics by comparing Silhouette Scores

In [ ]:
from sklearn.metrics import silhouette_score
from gensim.matutils import corpus2dense

def get_silhouette_score(df,feature):
  X = corpus2dense(df[feature].values, len(dictionary), num_docs=len(df)).T
  y = df['topic'].values
  return silhouette_score(X,y)

get_silhouette_score(df_2015,'body_bow')

In [ ]:
chunksize = 2000
passes = 20
iterations = 400
eval_every = None
temp = dictionary[0] # to 'load' dictionary
id2word = dictionary.id2token
feature = 'body_bow'

X = df_2015[feature].values
X_s = corpus2dense(X, len(dictionary), num_docs=len(X)).T

def argmax(ls):
  return max(ls, key = lambda item: item[1])


for num_topics in range(10,61):
  model = LdaModel(
    corpus=X,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=None
    )
  y_s = df_2015[feature].apply(lambda x : argmax(model.get_document_topics(x))[0]).values
  try:
    score = silhouette_score(X_s,y_s)
    print('Topic_size: {}, No. of labels: {}, Silhouette Score: {}'.format(num_topics, len(set(y_s)), score))
  except:
    print('Topic_size: {}, No. of labels: {},  Silhouette Score: {}'.format(num_topics, len(set(y_s)), -1))

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import matplotlib.colors as mcolors
%matplotlib inline

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'
cols = cols * 2

max_words = 15

cloud = WordCloud(background_color='white',
                  width=2500,
                  height=1800,
                  max_words=max_words,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = model.show_topics(formatted=False,num_topics=num_topics, num_words=max_words)

fig, axes = plt.subplots(5, 4, figsize=(20,20), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()